In [9]:
from google.genai import types
from google.adk.runners import InMemoryRunner
from google.adk.models.google_llm import Gemini
from google.adk.agents import Agent, LoopAgent
from google.adk.tools.tool_context import ToolContext

In [10]:
retry_config=types.HttpRetryOptions(attempts=3, exp_base=5, initial_delay=1, http_status_codes=[429, 500, 503, 504])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

In [15]:
idea_agent = Agent(
    name="IdeaAgent",
    model=model,
    instruction="""
    You are a creative idea generator for short videos for social media.
    Create unique and innovative idea based on user prompts.
    Return only one idea, concise and engaging, in a single sentence.
    Take this feedback into account: {feedback?}
    """,
    output_key="idea"
)

In [16]:
def exit_loop(tool_context: ToolContext) -> dict:
    """Call this function ONLY when the idea is approved, signaling the iterative process should end."""
    tool_context.actions.escalate = True
    return {}

validation_agent = Agent(
    name="ValidationAgent",
    model=model,
    instruction="""
    You are a video idea validator for short videos on social media targeted to kids.
    Based on the provided idea:
    - If the idea is appropriate for small children, you MUST call the 'exit_loop' function. Do not output any text.
    - Otherwise, you MUST respond with "INAPPROPRIATE" and provide a brief explanation why the idea is not suitable for kids.
    Idea: {idea}
    """,
    output_key="feedback",
    tools=[exit_loop]
)

In [17]:
orchiestrator_agent = LoopAgent(
    name="LoopPipeline",
    sub_agents=[idea_agent, validation_agent],
    max_iterations=3,
)

In [18]:
runner = InMemoryRunner(agent=orchiestrator_agent)
_ = await runner.run_debug("Create a short video about health risks.")


 ### Created new session: debug_session_id

User > Create a short video about health risks.
IdeaAgent > IdeaAgent: Imagine a "day in the life" video where everyday actions subtly reveal escalating health risks, ending with a stark visual of consequences.
ValidationAgent > INAPPROPRIATE: The idea of showing escalating health risks and stark visual consequences is not suitable for young children as it can be frightening and distressing.
IdeaAgent > IdeaAgent: Let's create a fun, animated video where friendly characters demonstrate simple healthy habits, like eating colorful fruits and vegetables and getting enough sleep, to show how they make us feel strong and energetic.
